In [34]:
import pandas as pd
import heapq
from functools import total_ordering

In [4]:
clients = pd.read_csv("DataSets\example-set\input_clients.csv")
display(clients)

,ClientID,Currencies,PositionCheck,Rating
0,A,"USD,SGD",Y,1
1,B,"USD,SGD,JPY",N,2
2,C,SGD,Y,3
3,D,USD,Y,4
4,E,SGD,N,5


In [36]:
instruments = pd.read_csv("DataSets\example-set\input_instruments.csv")
display(instruments)

,InstrumentID,Currency,LotSize
0,SIA,SGD,100


In [37]:
orders = pd.read_csv("DataSets\example-set\input_orders.csv")
display(orders)

,Time,OrderID,Instrument,Quantity,Client,Price,Side
0,9:00:01,A1,SIA,1500,A,Market,Buy
1,9:02:00,B1,SIA,4500,B,32.1,Sell
2,9:05:00,C1,SIA,100,C,32,Buy
3,9:10:00,D1,SIA,300,D,Market,Sell
4,9:29:01,B2,SIA,5,B,32.1,Sell
5,9:29:02,E1,SIA,1000,E,32,Sell
6,9:29:03,A2,SIA,800,A,31.9,Buy
7,9:30:01,C2,SIA,100,C,Market,Sell
8,9:40:00,B3,SIA,500,B,32.2,Sell
9,10:50:00,C3,SIA,4200,C,32.2,Buy


In [38]:
class Client:
    def __init__(self, ID, Currencies, PosCheck, Rating):
        self.ID = ID
        self.Currencies = Currencies
        self.PosCheck = PosCheck
        self.Rating = Rating
        self.NetPosition = {}

In [59]:

class Order:
    def __init__(self, Time, orderID, Instrument, Quantity, Client, Price, Side):
        self.Time =Time
        self.orderID = orderID
        self.Instrument = Instrument
        self.Quantity = Quantity
        self.Client = Client
        self.Price = Price
        self.Side = Side
        self.Rejection = False
        self.RejectionRsn = None
        self.ClientRating = None

    @total_ordering
    def __lt__(self, other):


        if (self.Price != other.Price):
            # Check for -1 for market order
            if (self.Price == -1): return True
            elif (other.Price == -1): return False
            if (self.Price < other.Price & self.Side == "Sell"):
                return True
            elif (self.Price > other.Price & self.Side == "Sell"):
                return False
            elif (self.Price > other.Price & self.Side == "Buy"):
                return True
            elif (self.Price < other.Price & self.Side == "Buy"):
                return False

        if (self.ClientRating > other.ClientRating):
            return True
        elif (self.ClientRating < other.ClientRating):
            return False
        elif (self.Time < other.Time):
            return True
        elif (self.Time >= other.Time):
            return False

    # def __gt__(self, other):
    #     if (self.Price > other.Price & self.Side == "Sell"):
    #         return True
    #     elif (self.Price < other.Price & self.Side == "Sell"):
    #         return False
    #     elif (self.Price < other.Price & self.Side == "Buy"):
    #         return True
    #     elif (self.Price > other.Price & self.Side == "Buy"):
    #         return False
    #     elif (self.ClientRating < other.ClientRating):
    #         return True
    #     elif (self.ClientRating > other.ClientRating):
    #         return False
    #     elif (self.Time > other.Time):
    #         return True
    #     elif (self.Time <= other.Time):
    #         return False

    def CheckOrder(self, Client, InstrumentNameList, InstrumentList):
        
        if (self.Instrument not in InstrumentNameList):
            #InstrumentNameList is just the InstrumentID column
            self.Rejection = True
            self.RejectionRsn = "REJECTED-INSTRUMENT NOT FOUND"
            return False
        else:
            for x in InstrumentList:
                if (x.ID == self.Instrument):
                    Instrument = x
        if (Instrument.Currency not in Client.Currencies):
            #REJECT MISMATCH CURRENCY
            self.Rejection = True
            self.RejectionRsn = "REJECTED-MISMATCH CURRENCY"
            return False
        elif (self.Quantity % Instrument.LotSize):
            self.Rejection = True
            self.RejectionRsn = "REJECTED-INVALID LOT SIZE"
            return False
        elif ((Client.PosCheck == "Y") & 
                (self.Side == "Sell") & 
                ((Client.NetPosition - Order.Quantity) < 0)):
            self.Rejection = True
            self.RejectionRsn = "REJECTED-POSITION CHECK FAILED"
            return False

        self.ClientRating = Client.Rating

        return True

In [66]:
class Instrument:
    def __init__(self, ID, Currency, LotSize):
        self.ID = ID
        self.Currency = Currency
        self.LotSize = LotSize
        self.OpenPrice = None
        self.ClosePrice = None
        self.TotalVolume = 0
        self.PricexVol = 0
        self.DayHigh = None
        self.DayLow = None
        self.sellheap = []
        self.buyheap = []
    def heappush(self, NewOrder):
        if (NewOrder.Side == "Sell"):
            heapq.heappush(self.sellheap, NewOrder)
        if (NewOrder.Side == "Buy"):
            heapq.heappush(self.buyheap, NewOrder)
    def heapread(self, side):
        if (side == "Buy"):
            return self.buyheap[0]
        else:
            return self.sellheap[0]
    def heappop(self, side):
        if (side == "Buy"):
            return heapq.heappop(self.buyheap)
        else:
            return heapq.heappop(self.sellheap)
    def isEmpty(self, side):
        if (side == "Buy"):
            return len(self.buyheap) == 0
        else:
            return len(self.sellheap) == 0



In [41]:
rejection = pd.DataFrame(columns = ['Order Id', 'Rejection Reason'])

In [42]:
def createClientList(clients):
    clientList = []
    for i in range(len(clients.index)):
        row = clients.iloc[i]
        client = Client(row['ClientID'], row['Currencies'], row['PositionCheck'], row['Rating'])
        clientList.append(client)
    return clientList

def findClient(clientId, clients):
    for client in clients:
        if client.clientID == clientId:
            return client

In [43]:
def createInstrumentList(instruments):
    instrumentList = []
    for i in range(len(instruments.index)):
        row = instruments.iloc[i]
        instrument = Instrument(row['InstrumentID'], row['Currency'], row['LotSize'])
        instrumentList.append(instrument)
    return instrumentList

In [44]:
clientList = createClientList(clients)
tickerHash = createInstrumentList(instruments)
orderList = []

In [45]:
def buildOrderBook(clients, instruments, orders, rejection):
    for i in range(len(clients.index)):
        row = orders.iloc[i]
    
    orders['Time'] = pd.to_datetime(orders['Time'], format="%H:%M:%S")

    for i in range(len(orders.index)):
        row = orders.iloc[i]
        if row['Price'] == 'Market':
            row['Price'] = -1

        order = Order(row['Time'], row['OrderID'], row['Instrument'], row['Quantity'], findClient(row['Client'], clients), row['Price'], row['Side'])
        client = Client()
        if row['Time'] < pd.to_datetime('9:30:00', format="%H:%M:%S"):
            if order.CheckOrder() == False:
                print("REJECT")
                rejection.append(order.orderID, order.RejectionRsn)
            else:
                orderList.append(order)
            
rejection = pd.DataFrame()
buildOrderBook(clients, instruments, orders, rejection)
rejection.to_csv('output_exchange_report.csv')

C:\Users\Hrishiraj\AppData\Local\Temp\ipykernel_5500\690892816.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Price'] = -1


AttributeError: 'str' object has no attribute 'clientID'

in()


In [80]:
##buildOrderBook()
orderList = [['9:00:01','A1','SIA','1500','A','Market','Buy']]
tickerHash = {"SIA":Instrument("SIA", "SGD", "100")}

clientHash = {"A": Client('A',"USD,SGD",'Y','1'),
              "B": Client('B',"USD,SGD,JPY",'N','2')}
def main():
    
    for orderRow in orderList:
        order = Order(orderRow[0], orderRow[1], orderRow[2], orderRow[3], orderRow[4], orderRow[5], orderRow[6])
        ticker = tickerHash[order.Instrument]
        ticker.heappush(order)
        hasBuy = not ticker.isEmpty("Buy")
        hasSell = not ticker.isEmpty("Sell")
        bestBuyOrder = None
        if (hasBuy):
            bestBuyOrder = ticker.heappop("Buy")
        bestSellOrder = None
        if (hasSell):
            bestSellOrder = ticker.heappop("Sell")
        
        
        if (hasBuy and hasSell):
            bestBuyOrderPrice = bestBuyOrder.price
            bestSellOrderPrice = bestSellOrder.price
            
            if (bestBuyOrderPrice >= bestSellOrderPrice):
                minVol = min(bestBuyOrder.volume, bestSellOrder.volume)
                bestBuyOrder.volume = bestBuyOrder.volume - minVol 
                bestSellOrder.volume = bestSellOrder.volume - minVol 
                maxPrice = max(bestBuyOrder.price, bestSellOrder.price)
                ticker.PricexVol = ticker.PricexVol + minVol * maxPrice
                ticker.TotalVolume = ticker.TotalVolume + minVol
                buyClient = bestBuyOrder.client
                sellClient = bestSellOrder.client

                buyClient.NetPosition = client.NetPosition - minVol * maxPrice
                sellClient.NetPosition = client.NetPosition - minVol * maxPrice
                clientHash[buyClient.Client] = buyClient
                clientHash[sellClient.Client] = sellClient

                if (bestBuyOrder.volume == 0):
                    ticker.heappush(bestSellOrder)
                else:
                    ticker.heappush(bestBuyOrder)
            else:
                
                ticker.heappush(bestBuyOrder)
                ticker.heappush(bestSellOrder)
        else:
            if (hasBuy):
                ticker.heappush(bestBuyOrder)
            else:
                ticker.heappush(bestSellOrder)
        

main()


NameError: name 'buildOrderBook' is not defined

In [ ]:
#Function to Match at 09:30

def Auction(Instrument, OoC, clientHash):
    matching = True
    currentPrice = 0
    volume = 0
    matchif = False
    InstrumentName = Instrument.ID
    if (Instrument.heapread('Buy')[-1].Price == -1):
        if (Instrument.heapread('Sell')[-1].Price == -1):
            currentPrice = "NULL"
            matchif = False
    
    while (matching):
        
        buyOrder = Instrument.heappop('Buy')
        sellOrder = Instrument.heappop('Sell')
        sellquant = sellOrder.Quantity
        buyquant = buyOrder.Quantity
        if (buyOrder.Price == -1 & sellOrder.Price == -1):
            matchif = True
            currentPrice = -1
        
        elif (buyOrder.Price == -1 & sellOrder.Price != -1):
            
            matchif = True
            if (sellOrder.Price > currentPrice): currentPrice = sellOrder.Price
        elif (buyOrder.Price != -1 & sellOrder.Price == -1):
            
            matchif = True
            if (sellOrder.Price < currentPrice): currentPrice = sellOrder.Price
        elif (buyOrder.Price >= sellOrder.Price):
            matchif = True
            currentPrice = buyOrder.Price

        if (matchif == False):
            matching = False
            continue
        else:
            if (buyquant > sellquant):
                sellquant = 0
                buyquant -= sellquant
                volume += sellquant
                 
            else:
                sellquant -= buyquant
                buyquant = 0
                volume += buyquant
        if (buyquant):
            buyOrder.Quantity = buyquant
            Instrument.heappush(buyOrder)
        if (sellquant):
            sellOrder.Quantity = sellquant
            Instrument.heappush(sellOrder)
    if (volume != 0):
        buyClient = clientHash[Instrument.buyOrder.Client]
        sellClient = clientHash[Instrument.sellOrder.Client]
        buyClient.NetPosition[InstrumentName] = buyClient.NetPosition[InstrumentName] + volume
        sellClient.NetPosition[InstrumentName] = sellClient.NetPosition[InstrumentName] - volume
    if (OoC == "Open"):
        Instrument.OpenPrice = currentPrice
    else:
        Instrument.ClosePrice = currentPrice
                
    if (currentPrice != "NULL"):
        Instrument.VolxPrice = volume * currentPrice
        Instrument.TotalVolume += volume
        

IndentationError: expected an indented block after 'if' statement on line 8 (1195572656.py, line 11)